<a href="https://colab.research.google.com/github/sanaz-mahmoudi/sanazmahmoudi/blob/main/Newcastle_University_Interview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pre Interview Task**

## Requirements

In [ ]:
# PYOMO
!pip install -q pyomo
import pyomo.environ as pe

# NUMPY
import numpy as np

# CBC
!apt-get install -y coinor-cbc
solver = pe.SolverFactory("cbc")

# PyPlot
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Set font configuration at the start
!wget -O TimesNewRoman.ttf \
  https://github.com/justrajdeep/fonts/raw/master/Times%20New%20Roman.ttf

import matplotlib.font_manager as fm
import matplotlib as mpl

fm.fontManager.addfont('TimesNewRoman.ttf')

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['Times New Roman']

# OTHER
import pandas as pd

## Input Data

In [ ]:
print('=====================================')
print('           Load Demand Data          ')
print('=====================================')
data_dem = [[0],[0],[100]]
nn = len(data_dem)
dem = pd.DataFrame(data_dem,index=['n'+str(i) for i in range(nn)],columns=['pmax'])
print(dem)

print('=====================================')
print('              Line Data              ')
print('=====================================')
data_lin = [[0,1,100],[1,2,100],[0,2,100]]
lin = pd.DataFrame(np.zeros((nn, nn), dtype=int), index=range(nn), columns=range(nn))
for from_bus, to_bus, pmax in data_lin:
    lin.loc[from_bus, to_bus] = pmax
    lin.loc[to_bus, from_bus] = pmax
print(lin)

print('=====================================')
print('       Thermal Power Plant Data      ')
print('=====================================')
data_gen = [[0,0,250,50]]
ng = len(data_gen)
gen = pd.DataFrame(data_gen,index=['g'+str(i) for i in range(ng)],columns=['bus','pmin','pmax','o_cost'])
print(gen)

print('=====================================')
print('      Sulphur-Flow Battery Data      ')
print('=====================================')
data_bat = [[1,100,0,2400,0,100,0,100,0.9,0.9]]
nb = len(data_bat)
bat = pd.DataFrame(data_bat,index=['b'+str(i) for i in range(nb)],
                   columns=['bus','eini','emin','emax','pcmin','pcmax','pdmin','pdmax','etac','etad'])
print(bat)

print('=====================================')
print('            Renewable Data           ')
print('=====================================')
data_ren = [[1,0,150,0]]
nw = len(data_ren)
ren = pd.DataFrame(data_ren,index=['w'+str(i) for i in range(nw)],columns=['bus','pmin','pmax','o_cost'])
print(ren)

print('=====================================')
print(' Hourly Demand and Renewable Profile ')
print('=====================================')
h_data = pd.read_csv('https://raw.githubusercontent.com/sanaz-mahmoudi/sanazmahmoudi/refs/heads/main/Hourly%20Profiles.csv')

nd = 28;  # Number of Days
nh = 24;  # Number of Hours

d_profile = h_data['Normalised Electricity Consumption Profile'].values.reshape((nd, nh))
pld = np.zeros((nd, nh, nn))
for i in range(nn):
    pld[:, :, i] = d_profile * dem['pmax'].values[i]
print('Hourly Load Demand =\n', pld)

r_profile = h_data['Normalised Electricity Consumption Profile'].values.reshape((nd, nh))
pr = np.zeros((nd, nh, nw))
for i in range(nw):
    pr[:, :, i] = r_profile * ren['pmax'].values[i]
print('Hourly Renewable Generstion =\n', pr)

print('=====================================')
print('     PyPlots for Hourly Profiles     ')
print('=====================================')
# Reshape into Groups of 7*24 (days*hours) Rows per Column
re_h_ren_data = h_data['Normalised Renewable Production Profile'].values.reshape(-1, 7*24).T
re_h_dem_data = h_data['Normalised Electricity Consumption Profile'].values.reshape(-1, 7*24).T

# Convert to DataFrame and Rename Columns
w_ren_data = pd.DataFrame(re_h_ren_data, columns=[f'week{i+1}' for i in range(re_h_ren_data.shape[1])])
print(w_ren_data)
w_dem_data = pd.DataFrame(re_h_dem_data, columns=[f'week{i+1}' for i in range(re_h_dem_data.shape[1])])
print(w_dem_data)

# -------------------- Plot 1: Renewable Production --------------------
colors = plt.cm.get_cmap('winter', w_ren_data.shape[1])

plt.figure(figsize=(10, 6))

x = np.arange(len(w_ren_data.index))

for i, week in enumerate(w_ren_data.columns):
    plt.plot(x, w_ren_data[week] * ren['pmax'].values, label=week, color=colors(i))

# Legend
legend_lines = [Line2D([0], [0], color=colors(i), lw=2) for i in range(w_ren_data.shape[1])]
legend_labels = [f'Week {i+1}' for i in range(w_ren_data.shape[1])]
plt.legend(legend_lines, legend_labels, loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=14)

# Axes
tick_positions = [1] + [24 * i for i in range(1, 8)]
tick_positions = [t for t in tick_positions if t <= 168]
plt.xticks(tick_positions, [str(t) for t in tick_positions], fontsize=12)
plt.yticks(fontsize=12)

# Labels
plt.xlabel('Hours (h)', fontname='Times New Roman', fontsize=18)
plt.ylabel('Renewable Production (MW)', fontname='Times New Roman', fontsize=18)
plt.title('Renewable Production Profile', fontname='Times New Roman', fontsize=18)
plt.tight_layout()
plt.show()

# -------------------- Plot 2: Electricity Consumption --------------------
# REDEFINE colormap, lines, labels for the second figure
colors = plt.cm.get_cmap('winter', w_dem_data.shape[1])

plt.figure(figsize=(10, 6))

x = np.arange(len(w_dem_data.index))

for i, week in enumerate(w_dem_data.columns):
    plt.plot(x, w_dem_data[week] * dem['pmax'].values[2], label=f'Week {i+1}', color=colors(i))

# Legend
legend_lines = [Line2D([0], [0], color=colors(i), lw=2) for i in range(w_dem_data.shape[1])]
legend_labels = [f'Week {i+1}' for i in range(w_dem_data.shape[1])]
plt.legend(legend_lines, legend_labels, loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=14)

# Axes
tick_positions = [1] + [24 * i for i in range(1, 8)]
tick_positions = [t for t in tick_positions if t <= 168]
plt.xticks(tick_positions, [str(t) for t in tick_positions], fontsize=12)
plt.yticks(fontsize=12)

# Labels
plt.xlabel('Hours (h)', fontname='Times New Roman', fontsize=18)
plt.ylabel('Electricity Consumption (MW)', fontname='Times New Roman', fontsize=18)
plt.title('Electricity Consumption Profile', fontname='Times New Roman', fontsize=18)
plt.tight_layout()
plt.show()


## Optimization Model

In [ ]:
# Model
m = pe.ConcreteModel()

# Sets
m.b  = pe.Set(initialize=list(range(nb)))                        # Set of Sulphur-Flow Batteries
m.d  = pe.Set(initialize=list(range(nd)))                        # Set of Load Demand Centers
m.g  = pe.Set(initialize=gen.index.tolist())                     # Set of Thermal Power Plants
m.n  = pe.Set(initialize=list(range(nn)))                        # Set of Nodes
m.np = pe.Set(initialize=list(range(nn)))                        # Set of Nodes
m.td = pe.Set(initialize=list(range(nd)))                        # Set of Days
m.th = pe.Set(initialize=list(range(nh)))                        # Set of Hours
m.w  = pe.Set(initialize=list(range(nw)))                        # Set of Renewable Sources

# Variables
m.bcb = pe.Var(m.b,m.td,m.th,within=pe.Binary)                   # Charging Status of Sulphur-Flow Batteries
m.bdb = pe.Var(m.b,m.td,m.th,within=pe.Binary)                   # Discharging Status of Sulphur-Flow Batteries
m.pcb = pe.Var(m.b,m.td,m.th,within=pe.NonNegativeReals)         # Power Charging of Sulphur-Flow Batteries
m.pdb = pe.Var(m.b,m.td,m.th,within=pe.NonNegativeReals)         # Power Discharging of Sulphur-Flow Batteries
m.pds = pe.Var(m.n,m.td,m.th,within=pe.Binary)                   # Demand Shedding
m.pg  = pe.Var(m.g,m.td,m.th,within=pe.NonNegativeReals)         # Power Production of Thermal Power Plants
m.pl  = pe.Var(m.n,m.np,m.td,m.th,within=pe.Reals)               # Power Flow of Lines
m.prs = pe.Var(m.w,m.td,m.th,within=pe.NonNegativeReals)         # Renewable Shedding
m.pw  = pe.Var(m.w,m.td,m.th,within=pe.NonNegativeReals)         # Power Production of Renewable Sources

# Objective function
def obj_rule(m):
  return sum(gen['o_cost'][g]*m.pg[g,td,th] for g in m.g for td in m.td for th in m.th)
m.obj = pe.Objective(rule=obj_rule)

# Constraints
# Energy balance
def bal_rule(m,n,td,th):
  return (sum(m.pg[g,td,th] for g in m.g if gen['bus'][g] == n) +
          sum(m.pw[w,td,th] for w in m.w if ren['bus'][w] == n) +
          sum(m.pdb[b,td,th] for b in m.b if bat['bus'][b] == n)) == (pld[td,th,n] +
                                                                      sum(m.pl[n,np,td,th] for np in m.np) +
                                                                      sum(m.pcb[b,td,th] for b in m.b if bat['bus'][b] == n))

m.bal = pe.Constraint(m.n, m.td, m.th, rule=bal_rule)

# ------------------- Thermal Power Plants --------------------
# Minimum Thermal Generation Capacity
def min_gen_rule(m,g,td,th):
    return m.pg[g,td,th] >= gen['pmin'][g]
m.min_gen = pe.Constraint(m.g, m.td, m.th, rule=min_gen_rule)

# Maximum Thermal Generation Capacity
def max_gen_rule(m,g,td,th):
    return m.pg[g,td,th] <= gen['pmax'][g]
m.max_gen = pe.Constraint(m.g, m.td,  m.th, rule=max_gen_rule)

# --------------------- Renewable Sources ---------------------
# Maximum Renewable Generation Capacity
def max_ren_rule(m,w,td,th):
    return m.pw[w,td,th] <= pr[td,th,w]
m.max_ren = pe.Constraint(m.w, m.td, m.th, rule=max_ren_rule)

# ------------------- Sulphur-Flow Battery --------------------
# Minimum Charging Limit of Sulphur-Flow Battery
def min_cha_rule(m,b,td,th):
    return m.pcb[b,td,th] >= bat['pcmin'][b]*m.bcb[b,td,th]
m.min_cha = pe.Constraint(m.b, m.td, m.th, rule=min_cha_rule)

# Maximum Charging Limit of Sulphur-Flow Battery
def max_cha_rule(m,b,td,th):
    return m.pcb[b,td,th] <= bat['pcmax'][b]*m.bcb[b,td,th]
m.max_cha = pe.Constraint(m.b, m.td, m.th, rule=max_cha_rule)

# Minimum Discharging Limit of Sulphur-Flow Battery
def min_dis_rule(m,b,td,th):
    return m.pdb[b,td,th] >= bat['pdmin'][b]*m.bdb[b,td,th]
m.min_dis = pe.Constraint(m.b, m.td, m.th, rule=min_dis_rule)

# Maximum Discharging Limit of Sulphur-Flow Battery
def max_dis_rule(m,b,td,th):
    return m.pdb[b,td,th] <= bat['pdmax'][b]*m.bdb[b,td,th]
m.max_dis = pe.Constraint(m.b, m.td, m.th, rule=max_dis_rule)

# Minimum Stored Energy of Sulphur-Flow Battery
def min_sto_rule(m,b,td):
    return bat['emin'][b] <= (bat['eini'][b] +
                              sum(bat['etac'][b]*m.pcb[b,td,th] - (1/bat['etad'][b])*m.pdb[b,td,th] for th in m.th))
m.min_sto = pe.Constraint(m.b, m.td, rule=min_sto_rule)

# Maximum Stored Energy of Sulphur-Flow Battery
def max_sto_rule(m,b,td):
    return bat['eini'][b] + sum(bat['etac'][b]*m.pcb[b,td,th] - (1/bat['etad'][b])*m.pdb[b,td,th] for th in m.th) <= bat['emax'][b]
m.max_sto = pe.Constraint(m.b, m.td, rule=max_sto_rule)

# Binding Initial and Final Stored Energy Levels of Sulphur-Flow Battery
def bid_sto_rule(m,b,td):
    return sum(bat['etac'][b]*m.pcb[b,td,th] - (1/bat['etad'][b])*m.pdb[b,td,th] for th in m.th) == 0
m.bid_sto = pe.Constraint(m.b, m.td, rule=bid_sto_rule)

# No Simultaneously Charges and Discharges
def cha_dis_rule(m,b,td,th):
    return m.bcb[b,td,th] + m.bdb[b,td,th] <= 1
m.cha_dis = pe.Constraint(m.b, m.td, m.th, rule=cha_dis_rule)

# -------------------------- Lines ----------------------------
# Minimum Line Flow
def min_flow_rule(m,n,np,td,th):
    return m.pl[n,np,td,th] >= -lin.loc[n,np]
m.min_flow = pe.Constraint(m.n, m.np, m.td, m.th, rule=min_flow_rule)

# Maximum Line Flow
def max_flow_rule(m,n,np,td,th):
    return m.pl[n,np,td,th] <= lin.loc[n,np]
m.max_flow = pe.Constraint(m.n, m.np, m.td, m.th, rule=max_flow_rule)

# Adjust Line Flow through Lines
def adj_flow_rule(m,n,np,td,th):
    return m.pl[n,np,td,th] == -m.pl[np,n,td,th]
m.adj_flow = pe.Constraint(m.n,m.np,m.td,m.th,rule=adj_flow_rule)

# Solve problem using GLPK solver
solver.options['mipgap'] = 1e-5
solver.solve(m,tee=True).write()

## Print Results

In [ ]:
# Objective Function
print('Optimal Value =',m.obj())

# Thermal Units
data = []

for (g, td, th) in m.pg:
            value = m.pg[g,td,th].value
            data.append({
                'g':g,
                'td': td,
                'th': th,
                'value': value
                })

df = pd.DataFrame(data)
df = df.sort_values(by=['g', 'td', 'th'])
thermal = df['g'].unique()

for g in thermal:
    sub_df = df[df['g'] == g]
    pivot = sub_df.pivot(index='td', columns='th', values='value')
    print(f"\nThermal Generation: {g}")
    display(pivot)

# Renewable Units
data = []

for (w, td, th) in m.pw:
            value = m.pw[w,td,th].value
            data.append({
                'w':w,
                'td': td,
                'th': th,
                'value': value
                })

df = pd.DataFrame(data)
df = df.sort_values(by=['w', 'td', 'th'])
renewable = df['w'].unique()

for w in renewable:
    sub_df = df[df['w'] == w]
    pivot = sub_df.pivot(index='td', columns='th', values='value')
    print(f"\nRenewable Generation: {w}")
    display(pivot)

# Charged Batteries
data = []

for (b, td, th) in m.pcb:
            value = m.pcb[b,td,th].value
            data.append({
                'b':b,
                'td': td,
                'th': th,
                'value': value
                })

df = pd.DataFrame(data)
df = df.sort_values(by=['b', 'td', 'th'])
batteries = df['b'].unique()

for b in batteries:
    sub_df = df[df['b'] == b]
    pivot = sub_df.pivot(index='td', columns='th', values='value')
    print(f"\nCharging Batteries: {b}")
    display(pivot)

# Discharged Batteries
data = []

for (b, td, th) in m.pdb:
            value = m.pdb[b,td,th].value
            data.append({
                'b':b,
                'td': td,
                'th': th,
                'value': value
                })

df = pd.DataFrame(data)
df = df.sort_values(by=['b', 'td', 'th'])
batteries = df['b'].unique()

for b in batteries:
    sub_df = df[df['b'] == b]
    pivot = sub_df.pivot(index='td', columns='th', values='value')
    print(f"\nDischarging Batteries: {b}")
    display(pivot)

# Lines
data = []

for (n, np, td, th) in m.pl:
    value = m.pl[n,np,td,th].value
    data.append({
        'n': n,
        'np': np,
        'td': td,
        'th': th,
        'value': value
        })

df = pd.DataFrame(data)
df = df.sort_values(by=['n', 'np', 'td', 'th'])

df['group'] = df['n'].astype(str) + ' → ' + df['np'].astype(str)

pivoted_tables = {}
groups = df['group'].unique()

for group in groups:
    sub_df = df[df['group'] == group]
    pivot = sub_df.pivot(index='td', columns='th', values='value')
    pivoted_tables[group] = pivot

for group, table in pivoted_tables.items():
    print(f"\n=== {group} ===")
    display(table)
